In [31]:
import pandas as pd

In [32]:
from textblob import TextBlob

In [33]:
import spacy

In [34]:
import yake

In [35]:
from spacy.pipeline.textcat import DEFAULT_SINGLE_TEXTCAT_MODEL

In [36]:
from nltk.corpus import stopwords

In [37]:
import numpy
import ast
import re

In [38]:
df = pd.read_csv('../../data/extracted_features_round4.csv', header=0)
del df['Unnamed: 0']
df['title_vectors_bert'] = df['title_vectors_bert'].apply(lambda x: ast.literal_eval(','.join(re.sub(r'(?<=\d)(\s+)(?=-?\d)', ',', x).splitlines())))
df['article_sentence_vectors_bert'] = df['article_sentence_vectors_bert'].apply(lambda x: ast.literal_eval(re.sub(r'((array\()|(dtype=float32\),)|(dtype=float32\)))', '', x)))

In [39]:
df['TL'] = df['title'].str.len()

In [40]:
df['AL'] = df['article'].str.len()

In [41]:
nlp = spacy.load('en_core_web_md')

In [42]:
df['title_tokens_spacy'] = df['title'].apply(lambda x: nlp(x))
df['article_tokens_spacy'] = df['article'].apply(lambda x: nlp(x))
df['title_lemmas_spacy'] = df.apply(lambda x: [token.lemma_ for token in x['title_tokens_spacy']], axis=1)
df['article_lemmas_spacy'] = df.apply(lambda x: [token.lemma_ for token in x['article_tokens_spacy']], axis=1)
df['title_tokens_no_stop_spacy'] = df['title'].apply(lambda x: nlp(' '.join([str(t) for t in nlp(x) if not t.is_stop])))
df['article_tokens_no_stop_spacy'] = df['article'].apply(lambda x: nlp(' '.join([str(t) for t in nlp(x) if not t.is_stop])))

In [43]:
def getWordCount(text):
    return len(TextBlob(text).words)

In [44]:
df['TWC'] = df['title'].apply(getWordCount)

In [45]:
df['AWC'] = df['article'].apply(getWordCount)

In [46]:
def getSentenceCount(text):
    return len(TextBlob(text).sentences)

In [47]:
df['TSC'] = df['title'].apply(getSentenceCount)

In [48]:
df['ASC'] = df['article'].apply(getSentenceCount)

In [49]:
df['TALRATIO'] = df['TL'] / df['AL']

In [50]:
df['TAWCRATIO'] = df['TWC'] / df['AWC']

In [51]:
df['TASCRATIO'] = df['TSC'] / df['ASC']

In [52]:
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [53]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

In [54]:
def getPartitionPolarity(text, partition_number, partition_index):
    start_index = len(text)//partition_number * (partition_index - 1)
    end_index = len(text)//partition_number * partition_index

    while start_index >= 0 and start_index < len(text) and text[start_index] != ' ':
        start_index = start_index + 1
    while end_index < len(text) and text[end_index] != ' ':
        end_index = end_index + 1
        
    return TextBlob(text[start_index:end_index]).sentiment.polarity

In [55]:
def getPartitionSubjectivity(text, partition_number, partition_index):
    start_index = len(text)//partition_number * (partition_index - 1)
    end_index = len(text)//partition_number * partition_index

    while start_index >= 0 and start_index < len(text) and text[start_index] != ' ':
        start_index = start_index + 1
    while end_index < len(text) and text[end_index] != ' ':
        end_index = end_index + 1
        
    return TextBlob(text[start_index:end_index]).sentiment.subjectivity

In [56]:
df['TP'] = df['title'].apply(getPolarity)

In [57]:
df['AP'] = df['article'].apply(getPolarity)

In [58]:
df['TAPD'] = df['AP'] - df['TP']

In [59]:
df['TAPMEAN'] = (df['AP'] + df['TP']) / 2

In [60]:
df['TS'] = df['title'].apply(getSubjectivity)

In [61]:
df['AS'] = df['article'].apply(getSubjectivity)

In [62]:
df['TASD'] = df['AS'] - df['TS']

In [63]:
df['TASMEAN'] = df['AS'] + df['TS']

In [265]:
df.to_csv('../../data/extracted_features_round1.csv')
training_set = df.drop(columns=['index', 'title', 'article', 'title_tokens_spacy', 'article_tokens_spacy', 'title_lemmas_spacy', 'article_lemmas_spacy', 'title_tokens_no_stop_spacy', 'article_tokens_no_stop_spacy'])
training_set = training_set.rename(columns={'label':'CV'})
training_set.to_csv('../../data/extracted_features_round1_training.csv')

In [64]:
def count_special_character(string, divisor=1, from_start=True): 
    if from_start:
        string = string[:-(len(string)//divisor)]
    else:
        string = string[len(string)//divisor:]
    special_char= 0
   
    for i in range(0, len(string)):  
        if (string[i].isalpha()):  
            continue

        elif (string[i].isdigit()):
            continue
            
        else: 
            special_char += 1
            
    return special_char

In [65]:
df['TSCC'] = df['title'].apply(count_special_character, args=(4, False)) / df['TL']

In [66]:
df['ASCC'] = df['article'].apply(count_special_character, args=(6,)) / df['AL']

In [67]:
df['TASCCRATIO'] = df['TSCC'] / df['ASCC']

In [68]:
df['TASCCD'] = df['ASCC'] - df['TSCC']

In [29]:
df

,index,title,article,label,TL,AL,title_tokens_spacy,article_tokens_spacy,title_lemmas_spacy,article_lemmas_spacy,...,TAMKWFP,TAKWSIMSPACY,TAKWSP,TAKWSPD,TAKWSS,TAKWSSD,TAKWSIMVARSPACY,TAGSSIMDISTSPACY,TSCC,ASCC
0,1305,"Nikki Bella Now Believes John Cena Wants Kids,...",John Cena's very public campaign to win back N...,4.5,68,1537,"(Nikki, Bella, Now, Believes, John, Cena, Want...","(John, Cena, 's, very, public, campaign, to, w...","[Nikki, Bella, now, believe, John, Cena, want,...","[John, Cena, 's, very, public, campaign, to, w...",...,0.0,0,0,0.0,0,0.0,0,-1.533333,0.132353,0.189980
1,6528,Rouhani says Iran will keep producing missiles...,DUBAI (Reuters) - Iran will continue to produc...,3.0,64,3753,"(Rouhani, says, Iran, will, keep, producing, m...","(DUBAI, (, Reuters, ), -, Iran, will, continue...","[Rouhani, say, Iran, will, keep, produce, miss...","[DUBAI, (, Reuters, ), -, Iran, will, continue...",...,0.0,0,0,0.0,0,0.0,0,-0.444444,0.125000,0.156142
2,13823,Jerry Jones: Ezekiel Elliott Won't Be Suspended,Jerry Jones tells TMZ Sports he does not belie...,4.5,47,686,"(Jerry, Jones, :, Ezekiel, Elliott, Wo, n't, B...","(Jerry, Jones, tells, TMZ, Sports, he, does, n...","[Jerry, Jones, :, Ezekiel, Elliott, will, not,...","[Jerry, Jones, tell, TMZ, Sports, he, do, not,...",...,0.0,0,0,0.0,0,0.0,0,-1.000000,0.148936,0.160350
3,29908,Sen. Marco Rubio just threatened to “take care...,The decorum of the United States Senate was on...,3.5,73,2788,"(Sen., Marco, Rubio, just, threatened, to, “, ...","(The, decorum, of, the, United, States, Senate...","[Sen., Marco, Rubio, just, threaten, to, "", ta...","[the, decorum, of, the, United, States, Senate...",...,0.0,0,0,0.0,0,0.0,0,-4.047619,0.164384,0.163917
4,31325,Darius McCrary Says Estranged Wife's Out for F...,"Former ""Family Matters"" star Darius McCrary to...",3.0,67,730,"(Darius, McCrary, Says, Estranged, Wife, 's, O...","(Former, "", Family, Matters, "", star, Darius, ...","[Darius, McCrary, say, estranged, Wife, 's, ou...","[former, "", Family, matter, "", star, Darius, M...",...,0.0,0,0,0.0,0,0.0,0,-0.500000,0.134328,0.176712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,2678751,Stimulus bill: When will you see money and ben...,(CNN)Congress and President Donald Trump have...,3.5,52,7393,"(Stimulus, bill, :, When, will, you, see, mone...","( , (, CNN)Congress, and, President, Donald, T...","[stimulus, bill, :, when, will, you, see, mone...","[ , (, cnn)congress, and, President, Donald, T...",...,0.0,0,0,0.0,0,0.0,0,-2.333333,0.173077,0.161775
963,2679327,California coronavirus: Doctors 'cautiously ho...,(CNN)Two weeks after San Francisco issued the...,4.0,95,3193,"(California, coronavirus, :, Doctors, ', cauti...","( , (, CNN)Two, weeks, after, San, Francisco, ...","[California, coronavirus, :, doctor, ', cautio...","[ , (, cnn)two, week, after, San, Francisco, i...",...,0.0,0,0,0.0,0,0.0,0,-2.333333,0.126316,0.160977
964,2683873,$4.3 Million Homes in California,What you Get A French Caribbean-inspired home ...,4.0,32,8775,"($, 4.3, Million, Homes, in, California)","(What, you, Get, A, French, Caribbean, -, insp...","[$, 4.3, million, Homes, in, California]","[what, you, get, a, french, Caribbean, -, insp...",...,0.0,0,0,0.0,0,0.0,0,0.943662,0.093750,0.166724
965,2684134,"Ventilators and Coronavirus: Amid Desperation,...",President Trump wants the private sector to fi...,2.5,82,9977,"(Ventilators, and, Coronavirus, :, Amid, Despe...","(President, Trump, wants, the, private, sector...","[ventilator, and, Coronavirus, :, amid, desper...","[President, Trump, want, the, private, sector,...",...,0.0,0,0,0.0,0,0.0,0,-5.347826,0.109756,0.157863


In [70]:
kw_extractor = yake.KeywordExtractor()
language = 'en'
max_ngram_size = 1
deduplication_threshold = 0.9
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, features=None)

In [71]:
df['title_keywords'] = df['title'].apply(lambda x: custom_kw_extractor.extract_keywords(x))

In [72]:
df['article_keywords'] = df['article'].apply(lambda x: custom_kw_extractor.extract_keywords(x))

In [73]:
df['TKWC'] = df['title_keywords'].apply(lambda x: len(x)) / df['TWC']

In [74]:
df['AKWC'] = df['article_keywords'].apply(lambda x: len(x)) / df['AWC']

In [75]:
df['TAKWCRATIO'] = df['TKWC'] / df['AKWC']

In [76]:
def get_most_important_keyword_polarity(keywords):
    if len(keywords) == 0:
        return 0
    
    max_importance = min(number for word, number in keywords)
    max_word = next((word for word, number in keywords if number == max_importance))
    return TextBlob(max_word).sentiment.polarity

In [77]:
df['TMIKWP'] = df['title_keywords'].apply(get_most_important_keyword_polarity)

In [78]:
df['AMIKWP'] = df['article_keywords'].apply(get_most_important_keyword_polarity)

In [79]:
def get_most_important_keyword_subjectivity(keywords):
    if len(keywords) == 0:
        return 0
    
    max_importance = min(number for word, number in keywords)
    max_word = next((word for word, number in keywords if number == max_importance))
    return TextBlob(max_word).sentiment.subjectivity

In [80]:
df['TMIKWS'] = df['title_keywords'].apply(get_most_important_keyword_subjectivity)

In [81]:
df['AMIKWS'] = df['article_keywords'].apply(get_most_important_keyword_subjectivity)

In [82]:
def get_first_sentence_polarity(text):
    return TextBlob(text).sentences[0].sentiment.polarity

In [83]:
df['AFSP'] = df['article'].apply(get_first_sentence_polarity)

In [84]:
df['TAFSPMEAN'] = (df['TP'] + df['AFSP']) / 2

In [85]:
df['TAFSPD'] = df['AFSP'] - df['TP']

In [86]:
def get_first_sentence_subjectivity(text):
    return TextBlob(text).sentences[0].sentiment.subjectivity

In [87]:
df['AFSS'] = df['article'].apply(get_first_sentence_subjectivity)

In [88]:
df['TAFSSMEAN'] = (df['TS'] + df['AFSS']) / 2

In [89]:
df['TAFSSD'] = df['AFSS'] - df['TP']

In [90]:
def get_most_important_keyword_occurrence(row):
    title_keywords = row['title_keywords']
    article_tokens = row['article_tokens_spacy']
    if len(title_keywords) == 0 or len(article_tokens) == 0:
        return 0
    
    title_max_importance = min(number for word, number in title_keywords)
    title_max_word = next((word for word, number in title_keywords if number == title_max_importance))
    title_max_lemma = nlp(title_max_word)[0].lemma_
    
    article_lemmas = row['article_lemmas_spacy']
    
    occurrences = 0

    for lemma in article_lemmas:
        if lemma.lower() == title_max_lemma.lower():
            occurrences = occurrences + 1
    return occurrences / len(article_lemmas)
    
    
    
    

In [91]:
df['TMIKWAORATIO'] = df.apply(get_most_important_keyword_occurrence, axis=1)

In [92]:
df['title_keyword_lemmas_spacy'] = df['title_keywords'].apply(lambda x: [(nlp(keyword)[0].lemma_, number) for keyword, number in x])

In [93]:
def get_title_keyword_occurrence(row):
    title_keywords = row['title_keywords']
    article_tokens = row['article_tokens_spacy']
    if len(title_keywords) == 0 or len(article_tokens) == 0:
        return 0

    title_lemmas = row['title_keyword_lemmas_spacy']
    
    article_lemmas = row['article_lemmas_spacy']
    
    occurrences = 0
    for title_lemma, title_number in title_lemmas:
        for article_lemma in article_lemmas:
            if article_lemma.lower() == title_lemma.lower():
                occurrences = occurrences + 1
    return occurrences / len(article_lemmas)
    

In [94]:
df['TKWAORATIO'] = df.apply(get_title_keyword_occurrence, axis=1)

In [95]:
data = df['TKWAORATIO'].to_numpy()
min_nonzero = numpy.min(data[numpy.nonzero(data)])
min_nonzero
df['TKWAORATIO'] = df['TKWAORATIO'].apply(lambda x: x if x > 0 else min_nonzero)
df['TKWAORATIO'] = df['TKWAORATIO'].apply(lambda x: numpy.where(x > 0.0000000001, numpy.log10(x), -10))

In [96]:
def get_title_keyword_occurrence_weighted(row):
    title_keywords = row['title_keywords']
    article_tokens = row['article_tokens_spacy']
    if len(title_keywords) == 0 or len(article_tokens) == 0:
        return 0

    title_lemmas = row['title_keyword_lemmas_spacy']
    
    article_lemmas = row['article_lemmas_spacy']
    
    occurrences = 0
    for title_lemma, title_number in title_lemmas:
        for article_lemma in article_lemmas:
            if article_lemma.lower() == title_lemma.lower():
                occurrences = occurrences + (1 * title_number)
    return occurrences / len(article_lemmas)

In [97]:
df['TKWAOWRATIO'] = df.apply(get_title_keyword_occurrence_weighted, axis=1)

In [98]:
data = df['TKWAOWRATIO'].to_numpy()
min_nonzero = numpy.min(data[numpy.nonzero(data)])
min_nonzero
df['TKWAOWRATIO'] = df['TKWAOWRATIO'].apply(lambda x: x if x > 0 else min_nonzero)
df['TKWAOWRATIO'] = df['TKWAOWRATIO'].apply(lambda x: numpy.where(x > 0.0000000001, numpy.log10(x), -10))

In [99]:
def get_mean_keyword_polarity(keywords):
    count = 0
    polarity = 0
    for word, number in keywords:
        polarity = polarity + TextBlob(word).sentiment.polarity
        count = count + 1
        
    if count != 0:
        return polarity / count
    
    return 0
        

In [100]:
df['TKWPMEAN'] = df['title_keywords'].apply(get_mean_keyword_polarity)

In [101]:
df['AKWPMEAN'] = df['article_keywords'].apply(get_mean_keyword_polarity)

In [102]:
df['TAKWPMEAN'] = (df['TKWPMEAN'] + df['AKWPMEAN']) / 2

In [103]:
df['TAKWPD'] = df['AKWPMEAN'] - df['TKWPMEAN']

In [104]:
def get_mean_keyword_polarity_weighted(keywords):
    count = 0
    polarity = 0
    for word, number in keywords:
        polarity = polarity + (TextBlob(word).sentiment.polarity * number)
        count = count + 1
        
    if count != 0:
        return polarity / count
    
    return 0

In [105]:
df['TKWPWMEAN'] = df['title_keywords'].apply(get_mean_keyword_polarity_weighted)
df['AKWPWMEAN'] = df['article_keywords'].apply(get_mean_keyword_polarity_weighted)
df['TAKWPWMEAN'] = (df['TKWPWMEAN'] + df['AKWPWMEAN']) / 2
df['TAKWPWD'] = df['AKWPWMEAN'] - df['TKWPWMEAN']

In [106]:
def get_most_important_keyword_first_position(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    article_lemmas = row['article_lemmas_spacy']
    if len(title_lemmas) == 0 or len(article_lemmas) == 0:
        return 1
    
    title_max_importance = min(number for word, number in title_lemmas)
    title_max_lemma = next((word for word, number in title_lemmas if number == title_max_importance))
    
    position = 0
    for lemma in article_lemmas:
        position = position + 1
        if lemma.lower() == title_max_lemma.lower():
            break
            
    return position / len(article_lemmas)

In [107]:
df['TAMIKWFP'] = df.apply(get_most_important_keyword_first_position, axis=1)

In [108]:
def get_most_important_keyword_distribution(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    article_lemmas = row['article_lemmas_spacy']
    if len(title_lemmas) == 0 or len(article_lemmas) == 0:
        return 0
    
    title_max_importance = min(number for word, number in title_lemmas)
    title_max_lemma = next((word for word, number in title_lemmas if number == title_max_importance))
    
    position = 0
    occurrences = []
    for lemma in article_lemmas:
        if lemma.lower() == title_max_lemma.lower():
            occurrences.append(position)
        position = position + 1
    
    if len(occurrences) == 0:
        return 0
    
    distribution = 0
    for position in occurrences:
        distribution = distribution + (position - (len(article_lemmas) - 1) / 2)
    
    distribution = distribution / len(occurrences)
            
    return distribution / ((len(article_lemmas) - 1) / 2)

In [109]:
df['TAMIKWDIST'] = df.apply(get_most_important_keyword_distribution, axis=1)

In [110]:
def get_keyword_distribution(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    article_lemmas = row['article_lemmas_spacy']
    if len(title_lemmas) == 0 or len(article_lemmas) == 0:
        return 0
    
    occurrences = []
    for title_lemma, title_number in title_lemmas:
        position = 0
        for article_lemma in article_lemmas:
            if article_lemma.lower() == title_lemma.lower():
                occurrences.append(position)
            position = position + 1
    
    if len(occurrences) == 0:
        return 0
    
    distribution = 0
    for position in occurrences:
        distribution = distribution + (position - (len(article_lemmas) - 1) / 2)
    
    distribution = distribution / len(occurrences)
            
    return distribution / ((len(article_lemmas) - 1) / 2)

In [111]:
df['TAKWDIST'] = df.apply(get_keyword_distribution, axis=1)

In [313]:
df.to_csv('../../data/extracted_features_round2.csv')
training_set = df.drop(columns=['index', 'title', 'article', 'title_keywords', 'article_keywords', 'title_tokens_spacy', 'article_tokens_spacy', 'title_lemmas_spacy', 'article_lemmas_spacy', 'title_keywords', 'article_keywords', 'title_keyword_lemmas_spacy', 'title_tokens_no_stop_spacy', 'article_tokens_no_stop_spacy'])
training_set = training_set.rename(columns={'label':'CV'})
training_set.to_csv('../../data/extracted_features_round2_training.csv')

In [112]:
def get_similarity_score(row):
    title_doc = row['title_tokens_no_stop_spacy']
    article_doc = row ['article_tokens_no_stop_spacy']
    
    return title_doc.similarity(article_doc)

In [113]:
df['TASIMSPACY'] = df.apply(get_similarity_score, axis=1)

In [114]:
df['article_sentences'] = df['article'].apply(lambda x: TextBlob(x).sentences)

In [115]:
df['article_sentence_tokens_spacy'] = df['article_sentences'].apply(lambda x: [nlp(str(sentence)) for sentence in x])

In [116]:
df['article_sentence_tokens_no_stop_spacy'] = df['article_sentence_tokens_spacy'].apply(lambda x: [nlp(' '.join([str(t) for t in sentence if not t.is_stop])) for sentence in x])

In [117]:
def get_best_sentence_similarity_score(row):
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
        
    scores = []
    for doc in article_docs:
        scores.append(title_doc.similarity(doc))
    
    return max(scores)

In [118]:
df['TABSSIMSPACY'] = df.apply(get_best_sentence_similarity_score, axis=1)

C:\Users\tobia\AppData\Local\Temp/ipykernel_38984/4069332995.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  scores.append(title_doc.similarity(doc))


In [119]:
def get_worst_sentence_similarity_score(row):
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
        
    scores = []
    for doc in article_docs:
        scores.append(title_doc.similarity(doc))
    
    return min(scores)

In [120]:
df['TAWSSIMSPACY'] = df.apply(get_worst_sentence_similarity_score, axis=1)

C:\Users\tobia\AppData\Local\Temp/ipykernel_38984/2013556861.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  scores.append(title_doc.similarity(doc))


In [121]:
def get_average_sentence_similarity_score(row):
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
        
    score = 0
    for doc in article_docs:
        score = score + (title_doc.similarity(doc))
    
    return score / len(article_docs)

In [122]:
df['TAASSIMSPACY'] = df.apply(get_average_sentence_similarity_score, axis=1)

C:\Users\tobia\AppData\Local\Temp/ipykernel_38984/2634829990.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  score = score + (title_doc.similarity(doc))


In [123]:
def get_first_sentence_similarity_score(row):
    title_doc = row['title_tokens_no_stop_spacy']
    article_doc = row['article_sentence_tokens_no_stop_spacy'][0]
    
    return title_doc.similarity(article_doc)

In [124]:
df['TAFSSIMSPACY'] = df.apply(get_first_sentence_similarity_score, axis=1)

In [125]:
def get_best_sentence_similarity_position(row):
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
        
    scores = []
    for doc in article_docs:
        scores.append(title_doc.similarity(doc))
    
    max_score = max(scores)
    
    return (scores.index(max_score) + 1) / len(article_docs)

In [126]:
df['TABSSIMPSPACY'] = df.apply(get_best_sentence_similarity_position, axis=1)

C:\Users\tobia\AppData\Local\Temp/ipykernel_38984/2584009954.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  scores.append(title_doc.similarity(doc))


In [127]:
def get_worst_sentence_similarity_position(row):
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
        
    scores = []
    for doc in article_docs:
        scores.append(title_doc.similarity(doc))
    
    min_score = min(scores)
    
    return (scores.index(min_score) + 1) / len(article_docs)

In [128]:
df['TAWSSIMPSPACY'] = df.apply(get_worst_sentence_similarity_position, axis=1)

C:\Users\tobia\AppData\Local\Temp/ipykernel_38984/3901347589.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  scores.append(title_doc.similarity(doc))


In [129]:
def get_good_sentence_similarity_ratio(row):
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
        
    scores = []
    for doc in article_docs:
        scores.append(title_doc.similarity(doc))
        
    condition = lambda x: x>=0.90
    
    return sum(condition(x) for x in scores) / len(article_docs)

In [130]:
df['TAGSSIMRATIOSPACY'] = df.apply(get_good_sentence_similarity_ratio, axis=1)

C:\Users\tobia\AppData\Local\Temp/ipykernel_38984/3374778989.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  scores.append(title_doc.similarity(doc))


In [131]:
def count_word_types(doc, word_type):
    count = 0
    
    for token in doc:
        if token.pos_ == word_type:
            count = count + 1
    
    return count / len(doc)

In [132]:
df['TVC'] = df['title_tokens_spacy'].apply(count_word_types, args=('VERB',))

In [133]:
df['AVC'] = df['article_tokens_spacy'].apply(count_word_types, args=('VERB',))

In [134]:
df['TAVCD'] = df['AVC'] - df['TVC']

In [135]:
df['TNC'] = df['title_tokens_spacy'].apply(count_word_types, args=('NOUN',))

In [136]:
df['ANC'] = df['article_tokens_spacy'].apply(count_word_types, args=('NOUN',))

In [137]:
df['TANCD'] = df['ANC'] - df['TNC']

In [340]:
df.to_csv('../../data/extracted_features_round3.csv')
training_set = df.drop(columns=['index', 'title', 'article', 'title_keywords', 'article_keywords', 'title_tokens_spacy', 'article_tokens_spacy', 'title_lemmas_spacy', 'article_lemmas_spacy', 'title_keywords', 'article_keywords', 'title_keyword_lemmas_spacy', 'article_sentences', 'article_sentence_tokens_spacy', 'title_tokens_no_stop_spacy', 'article_tokens_no_stop_spacy', 'article_sentence_tokens_no_stop_spacy'])
training_set = training_set.rename(columns={'label':'CV'})
training_set.to_csv('../../data/extracted_features_round3_training.csv')

In [142]:
import torch#pytorch
from transformers import AutoTokenizer, AutoModel#for embeddings
from sklearn.metrics.pairwise import cosine_similarity#for similarity

In [157]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased",)
bert_model = AutoModel.from_pretrained("bert-base-uncased", output_hidden_states=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [158]:
def get_embeddings_bert(text, token_length):
    tokens=bert_tokenizer(text,max_length=token_length,padding='max_length',truncation=True)
    output=bert_model(torch.tensor(tokens.input_ids).unsqueeze(0),
               attention_mask=torch.tensor(tokens.attention_mask).unsqueeze(0)).hidden_states[-1]
    return torch.mean(output,axis=1).detach().numpy()

In [159]:
df['title_vectors_bert'] = df['title'].apply(get_embeddings_bert, args=(256,))

In [163]:
df['article_sentence_vectors_bert'] = df['article_sentences'].apply(lambda x: [get_embeddings_bert(str(sentence), 256) for sentence in x])

In [190]:
def get_best_sentence_similarity_score_bert(row):
    title_vector = row['title_vectors_bert']
    article_vectors = row['article_sentence_vectors_bert']
        
    scores = []
    for vector in article_vectors:
        scores.append(cosine_similarity(title_vector, vector)[0][0])
    
    return max(scores)

In [191]:
df['TABSSIMBERT'] = df.apply(get_best_sentence_similarity_score_bert, axis=1)

In [192]:
def get_worst_sentence_similarity_score_bert(row):
    title_vector = row['title_vectors_bert']
    article_vectors = row['article_sentence_vectors_bert']
        
    scores = []
    for vector in article_vectors:
        scores.append(cosine_similarity(title_vector, vector)[0][0])
    
    return min(scores)

In [193]:
df['TAWSSIMBERT'] = df.apply(get_worst_sentence_similarity_score_bert, axis=1)

In [194]:
def get_average_sentence_similarity_score_bert(row):
    title_vector = row['title_vectors_bert']
    article_vectors = row['article_sentence_vectors_bert']
        
    score = 0
    for vector in article_vectors:
        score = score + cosine_similarity(title_vector, vector)[0][0]
    
    return score / len(article_vectors)

In [195]:
df['TAASSIMSBERT'] = df.apply(get_average_sentence_similarity_score_bert, axis=1)

In [196]:
def get_first_sentence_similarity_score_bert(row):
    title_vector = row['title_vectors_bert']
    article_vector = row['article_sentence_vectors_bert'][0]
    
    return cosine_similarity(title_vector, article_vector)[0][0]

In [197]:
df['TAFSSIMBERT'] = df.apply(get_first_sentence_similarity_score_bert, axis=1)

In [198]:
def get_best_sentence_similarity_position_bert(row):
    title_vector = row['title_vectors_bert']
    article_vectors = row['article_sentence_vectors_bert']
        
    scores = []
    for vector in article_vectors:
        scores.append(cosine_similarity(title_vector, vector)[0][0])
    
    max_score = max(scores)
    
    return (scores.index(max_score) + 1) / len(article_vectors)

In [199]:
df['TABSSIMPBERT'] = df.apply(get_best_sentence_similarity_position_bert, axis=1)

In [433]:
def get_worst_sentence_similarity_position_bert(row):
    title_vector = row['title_vectors_bert']
    article_vectors = row['article_sentence_vectors_bert']
    scores = []
    for vector in article_vectors:
        scores.append(cosine_similarity(title_vector, vector)[0][0])
    
    min_score = min(scores)
    
    return (scores.index(min_score) + 1) / len(article_vectors)

In [437]:
df['TAWSSIMPBERT'] = df.apply(get_worst_sentence_similarity_position_bert, axis=1)

In [396]:
def get_good_sentence_similarity_ratio_bert(row):
    title_keywords = row['title_keywords']
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
        
    scores = []
    for doc in article_docs:
        scores.append(title_doc.similarity(doc))
    
    max_score = max(scores)
    
    return (scores.index(max_score) + 1) / len(article_docs)

In [206]:
df['TAGSSIMRATIOBERT'] = df.apply(get_good_sentence_similarity_ratio_bert, axis=1)

In [138]:
def get_most_important_keyword_average_sentence_similarity_spacy(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
    
    if len(title_lemmas) == 0 or len(article_docs) == 0:
        return 1
    title_max_importance = min(number for word, number in title_lemmas)
    title_max_lemma = next((word for word, number in title_lemmas if number == title_max_importance))
    
    matching_article_docs = []
    for doc in article_docs:
        if title_max_lemma in [token.lemma_ for token in doc]:
            matching_article_docs.append(doc)
    
    score = 0
    for doc in matching_article_docs:
        score = score + (title_doc.similarity(doc))
    
    return score / len(article_docs)

In [139]:
df['TAMIKWASSIMSPACY'] = df.apply(get_most_important_keyword_average_sentence_similarity_spacy, axis=1)

In [140]:
def get_most_important_keyword_best_sentence_similarity_spacy(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
    
    if len(title_lemmas) == 0 or len(article_docs) == 0:
        return 1
    title_max_importance = min(number for word, number in title_lemmas)
    title_max_lemma = next((word for word, number in title_lemmas if number == title_max_importance))
    
    matching_article_docs = []
    for doc in article_docs:
        if title_max_lemma in [token.lemma_ for token in doc]:
            matching_article_docs.append(doc)
    
    scores = []
    for doc in matching_article_docs:
        scores.append(title_doc.similarity(doc))
    
    return max(scores) if len(scores) != 0 else 0.8

In [141]:
df['TAMIKWBSSIMSPACY'] = df.apply(get_most_important_keyword_best_sentence_similarity_spacy, axis=1)

In [142]:
def get_most_important_keyword_worst_sentence_similarity_spacy(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
    
    if len(title_lemmas) == 0 or len(article_docs) == 0:
        return 1
    title_max_importance = min(number for word, number in title_lemmas)
    title_max_lemma = next((word for word, number in title_lemmas if number == title_max_importance))
    
    matching_article_docs = []
    for doc in article_docs:
        if title_max_lemma in [token.lemma_ for token in doc]:
            matching_article_docs.append(doc)
    
    scores = []
    for doc in matching_article_docs:
        scores.append(title_doc.similarity(doc))
    
    return min(scores) if len(scores) != 0 else 0.7

In [143]:
df['TAMIKWWSSIMSPACY'] = df.apply(get_most_important_keyword_worst_sentence_similarity_spacy, axis=1)

In [144]:
def get_most_keywords_similarity(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
    
    max_count = 0
    sentences = []
    for doc in article_docs:
        count = 0
        for title_lemma in title_lemmas:
            if title_lemma in [token.lemma_ for token in doc]:
                count = count + 1
        if count > max_count:
            sentences = [doc]
        elif count == max_count:
            sentences.append(doc)
            
    sim_score = 0
    for sentence in sentences:
        sim_score = sim_score + title_doc.similarity(sentence)
    
    return sim_score / len(sentences) if len(sentences) != 0 else 0

In [145]:
df['TAMKWSSIMSPACY'] = df.apply(get_most_keywords_similarity, axis=1)

C:\Users\tobia\AppData\Local\Temp/ipykernel_38984/1740321506.py:20: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim_score = sim_score + title_doc.similarity(sentence)


In [146]:
def get_most_keywords_sentence_polarity(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
    article_sentences = row['article_sentences']
    
    max_count = 0
    sentences = []
    sentence_index = 0
    for doc in article_docs:
        count = 0
        for title_lemma in title_lemmas:
            if title_lemma in [token.lemma_ for token in doc]:
                count = count + 1
        if count > max_count:
            sentences = [sentence_index]
        elif count == max_count:
            sentences.append(sentence_index)
        sentence_index = sentence_index + 1
            
    pol_score = 0
    for sentence in sentences:
        pol_score = pol_score + TextBlob(str(article_sentences[sentence])).sentiment.polarity
    
    return pol_score / len(sentences) if len(sentences) != 0 else 0

In [147]:
df['TAMKWSP'] = df.apply(get_most_keywords_sentence_polarity, axis=1)

In [148]:
df['TAMKWSPD'] = df['TAMKWSP'] - df['TP']

In [149]:
def get_most_keywords_sentence_subjectivity(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
    article_sentences = row['article_sentences']
    
    max_count = 0
    sentences = []
    sentence_index = 0
    for doc in article_docs:
        count = 0
        for title_lemma in title_lemmas:
            if title_lemma in [token.lemma_ for token in doc]:
                count = count + 1
        if count > max_count:
            sentences = [sentence_index]
        elif count == max_count:
            sentences.append(sentence_index)
        sentence_index = sentence_index + 1
            
    pol_score = 0
    for sentence in sentences:
        pol_score = pol_score + TextBlob(str(article_sentences[sentence])).sentiment.subjectivity
    
    return pol_score / len(sentences) if len(sentences) != 0 else 0

In [150]:
df['TAMKWSS'] = df.apply(get_most_keywords_sentence_subjectivity, axis=1)

In [151]:
df['TAMKWSSD'] = df['TAMKWSS'] - df['TP']

In [152]:
def get_most_keywords_sentence_position(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
    article_sentences = row['article_sentences']
    
    max_count = 0
    best_sentence_index = 0
    sentence_index = 1
    for doc in article_docs:
        count = 0
        for title_lemma in title_lemmas:
            if title_lemma[0] in [token.lemma_ for token in doc]:
                count = count + 1
        if count > max_count:
            best_sentence_index = sentence_index
            max_count = count
        sentence_index = sentence_index + 1
            
    return best_sentence_index / len(article_sentences)

In [153]:
df['TAMKWFP'] = df.apply(get_most_keywords_sentence_position, axis=1)

In [154]:
def get_keyword_sentences_similarity(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
    
    sentences = []
    for doc in article_docs:
        count = 0
        for title_lemma in title_lemmas:
            if title_lemma[0] in [token.lemma_ for token in doc]:
                count = count + 1
        if count > 0:
            sentences.append(doc)
           
            
    sim_score = 0
    for sentence in sentences:
        sim_score = sim_score + title_doc.similarity(sentence)
    
    return sim_score / len(sentences) if len(sentences) != 0 else 0

In [155]:
df['TAKWSIMSPACY'] = df.apply(get_keyword_sentences_similarity, axis=1)

In [156]:
def get_keyword_sentences_polarity(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
    article_sentences = row['article_sentences']
    
    sentences = []
    sentence_index = 0
    for doc in article_docs:
        count = 0
        for title_lemma in title_lemmas:
            if title_lemma[0] in [token.lemma_ for token in doc]:
                count = count + 1
        if count > 0:
            sentences.append(sentence_index)
            
    pol_score = 0
    for sentence in sentences:
        pol_score = pol_score + TextBlob(str(article_sentences[sentence])).sentiment.polarity
    
    return pol_score / len(sentences) if len(sentences) != 0 else 0

In [157]:
df['TAKWSP'] = df.apply(get_keyword_sentences_polarity, axis=1)

In [158]:
df['TAKWSPD'] = df['TAKWSP'] - df['TP']

In [159]:
def get_keyword_sentences_subjectivity(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
    article_sentences = row['article_sentences']
    
    sentences = []
    sentence_index = 0
    for doc in article_docs:
        count = 0
        for title_lemma in title_lemmas:
            if title_lemma[0] in [token.lemma_ for token in doc]:
                count = count + 1
        if count > 1:
            sentences.append(sentence_index)
            
    pol_score = 0
    for sentence in sentences:
        pol_score = pol_score + TextBlob(str(article_sentences[sentence])).sentiment.subjectivity
    
    return pol_score / len(sentences) if len(sentences) else 0

In [160]:
df['TAKWSS'] = df.apply(get_keyword_sentences_polarity, axis=1)

In [161]:
df['TAKWSSD'] = df['TAKWSS'] - df['TS']

In [162]:
def get_keyword_sentences_similarity_variance(row):
    title_lemmas = row['title_keyword_lemmas_spacy']
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
    
    sentences = []
    for doc in article_docs:
        count = 0
        for title_lemma in title_lemmas:
            if title_lemma[0] in [token.lemma_ for token in doc]:
                count = count + 1
        if count > 0:
            sentences.append(doc)
           
            
    sim_scores = []
    for sentence in sentences:
        sim_score = sim_scores.append(title_doc.similarity(sentence))
    
    return numpy.var(sim_scores) / row['AL'] if len(sim_scores) != 0 else 0

In [163]:
df['TAKWSIMVARSPACY'] = df.apply(get_keyword_sentences_similarity_variance, axis=1)

In [164]:
def get_good_sentence_similarity_distribution(row):
    title_doc = row['title_tokens_no_stop_spacy']
    article_docs = row['article_sentence_tokens_no_stop_spacy']
    best_sim = row['TABSSIMSPACY']
        
    scores = []
    for doc in article_docs:
        scores.append(title_doc.similarity(doc))
        
    condition = lambda x: x>=0.90
    
    index = 0
    indexes = []
    for score in scores:
        if score >= (best_sim * 0.8):
            indexes.append(index)
        index = index + 1
    
    
    distribution = 0
    for i in indexes:
        distribution = distribution + (i - (len(article_docs) - 1) / 2)
            
    return distribution / ((len(article_docs) - 1) / 2) if ((len(article_docs) - 1) / 2) != 0 else distribution

In [165]:
df['TAGSSIMDISTSPACY'] = df.apply(get_good_sentence_similarity_distribution, axis=1)

C:\Users\tobia\AppData\Local\Temp/ipykernel_38984/1416477517.py:8: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  scores.append(title_doc.similarity(doc))


In [166]:
df

,index,title,article,label,TL,AL,title_tokens_spacy,article_tokens_spacy,title_lemmas_spacy,article_lemmas_spacy,...,TAMKWFP,TAKWSIMSPACY,TAKWSP,TAKWSPD,TAKWSS,TAKWSSD,TAKWSIMVARSPACY,TAGSSIMDISTSPACY,TSCC,ASCC
0,1305,"Nikki Bella Now Believes John Cena Wants Kids,...",John Cena's very public campaign to win back N...,4.5,68,1537,"(Nikki, Bella, Now, Believes, John, Cena, Want...","(John, Cena, 's, very, public, campaign, to, w...","[Nikki, Bella, now, believe, John, Cena, want,...","[John, Cena, 's, very, public, campaign, to, w...",...,0.062500,0.720282,0.175000,-0.025000,0.175000,0.075000,2.768216e-06,-1.533333,0.132353,0.189980
1,6528,Rouhani says Iran will keep producing missiles...,DUBAI (Reuters) - Iran will continue to produc...,3.0,64,3753,"(Rouhani, says, Iran, will, keep, producing, m...","(DUBAI, (, Reuters, ), -, Iran, will, continue...","[Rouhani, say, Iran, will, keep, produce, miss...","[DUBAI, (, Reuters, ), -, Iran, will, continue...",...,0.052632,0.794728,0.000000,0.000000,0.000000,0.000000,4.327873e-07,-0.444444,0.125000,0.156142
2,13823,Jerry Jones: Ezekiel Elliott Won't Be Suspended,Jerry Jones tells TMZ Sports he does not belie...,4.5,47,686,"(Jerry, Jones, :, Ezekiel, Elliott, Wo, n't, B...","(Jerry, Jones, tells, TMZ, Sports, he, does, n...","[Jerry, Jones, :, Ezekiel, Elliott, will, not,...","[Jerry, Jones, tell, TMZ, Sports, he, do, not,...",...,0.142857,0.601711,-0.142857,-0.142857,-0.142857,-0.142857,1.048783e-05,-1.000000,0.148936,0.160350
3,29908,Sen. Marco Rubio just threatened to “take care...,The decorum of the United States Senate was on...,3.5,73,2788,"(Sen., Marco, Rubio, just, threatened, to, “, ...","(The, decorum, of, the, United, States, Senate...","[Sen., Marco, Rubio, just, threaten, to, "", ta...","[the, decorum, of, the, United, States, Senate...",...,0.045455,0.747132,0.350000,0.350000,0.350000,0.350000,2.458884e-06,-4.047619,0.164384,0.163917
4,31325,Darius McCrary Says Estranged Wife's Out for F...,"Former ""Family Matters"" star Darius McCrary to...",3.0,67,730,"(Darius, McCrary, Says, Estranged, Wife, 's, O...","(Former, "", Family, Matters, "", star, Darius, ...","[Darius, McCrary, say, estranged, Wife, 's, ou...","[former, "", Family, matter, "", star, Darius, M...",...,0.200000,0.685036,0.000000,0.000000,0.000000,0.000000,9.459142e-06,-0.500000,0.134328,0.176712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,2678751,Stimulus bill: When will you see money and ben...,(CNN)Congress and President Donald Trump have...,3.5,52,7393,"(Stimulus, bill, :, When, will, you, see, mone...","( , (, CNN)Congress, and, President, Donald, T...","[stimulus, bill, :, when, will, you, see, mone...","[ , (, cnn)congress, and, President, Donald, T...",...,0.015625,0.756684,-0.077778,-0.077778,-0.077778,-0.077778,2.291965e-07,-2.333333,0.173077,0.161775
963,2679327,California coronavirus: Doctors 'cautiously ho...,(CNN)Two weeks after San Francisco issued the...,4.0,95,3193,"(California, coronavirus, :, Doctors, ', cauti...","( , (, CNN)Two, weeks, after, San, Francisco, ...","[California, coronavirus, :, doctor, ', cautio...","[ , (, cnn)two, week, after, San, Francisco, i...",...,0.040000,0.753246,0.175000,0.075000,0.175000,-0.125000,1.137096e-06,-2.333333,0.126316,0.160977
964,2683873,$4.3 Million Homes in California,What you Get A French Caribbean-inspired home ...,4.0,32,8775,"($, 4.3, Million, Homes, in, California)","(What, you, Get, A, French, Caribbean, -, insp...","[$, 4.3, million, Homes, in, California]","[what, you, get, a, french, Caribbean, -, insp...",...,0.013889,0.520264,0.300000,0.300000,0.300000,0.300000,3.935128e-07,0.943662,0.093750,0.166724
965,2684134,"Ventilators and Coronavirus: Amid Desperation,...",President Trump wants the private sector to fi...,2.5,82,9977,"(Ventilators, and, Coronavirus, :, Amid, Despe...","(President, Trump, wants, the, private, sector...","[ventilator, and, Coronavirus, :, amid, desper...","[President, T

In [167]:
df.to_csv('../../data/extracted_features_round4.csv')

In [168]:

training_set = df.drop(columns=['index', 'title', 'article', 'title_keywords', 'article_keywords', 'title_tokens_spacy', 'article_tokens_spacy', 'title_lemmas_spacy', 'article_lemmas_spacy', 'title_keywords', 'article_keywords', 'title_keyword_lemmas_spacy', 'article_sentences', 'article_sentence_tokens_spacy', 'title_tokens_no_stop_spacy', 'article_tokens_no_stop_spacy', 'article_sentence_tokens_no_stop_spacy', 'title_vectors_bert', 'article_sentence_vectors_bert'])
training_set = training_set.rename(columns={'label':'CV'})
training_set.to_csv('../../data/extracted_features_round4_training.csv')